<center><h1>RESUME EVALUATOR AI</h1></center>

In [4]:
import fitz
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer

#1. Extract Text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text.strip()
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

#2. Summarize CV content
def summarize_cv(text):
    try:
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)
        summary = summarizer(text[:1024], max_length=150, min_length=40, do_sample=False)[0]['summary_text']
        return summary
    except Exception as e:
        print("Summarization Error:", e)
        return "Summary not available."

#3. Extract keywords from text
def get_keywords(text, top_n=10):
    try:
        vectorizer = CountVectorizer(stop_words='english')
        X = vectorizer.fit_transform([text])
        if not vectorizer.vocabulary_:
            return [("No keywords found", 0)]
        word_freq = X.toarray().sum(axis=0)
        vocab = vectorizer.get_feature_names_out()
        keywords = [(vocab[i], word_freq[i]) for i in range(len(vocab))]
        keywords = sorted(keywords, key=lambda x: x[1], reverse=True)
        return keywords[:top_n]
    except Exception as e:
        print("Keyword Extraction Error:", e)
        return [("Error", 0)]

#4. Score the Resume
def score_resume(text, summary, keywords):
    score = 0
    suggestions = []

    #Text extraction check
    if len(text) > 100:
        score += 20
    else:
        suggestions.append("Text seems too short or empty.")

    #Summary length check
    if len(summary.split()) >= 40:
        score += 30
    else:
        suggestions.append("Summary is too short. Add more content.")

    #Keyword richness check
    keyword_count = len([kw for kw in keywords if kw[1] > 1])
    if keyword_count >= 5:
        score += 30
    else:
        suggestions.append("Include more relevant keywords or skills.")

    #Education or experience check
    if any(term in text.lower() for term in ["education", "experience", "bachelor", "internship", "project"]):
        score += 20
    else:
        suggestions.append("Mention your education or professional experience.")

    return score, suggestions

#5. Main Analyzer Function
def analyze_cv(pdf_path):
    print("\nAnalyzing Resume:")

    #Extract and process
    text = extract_text_from_pdf(pdf_path)
    if not text:
        print("No valid text found in resume.")
        return

    summary = summarize_cv(text)
    keywords = get_keywords(text)

    # Score and Suggestions
    score, suggestions = score_resume(text, summary, keywords)

    print("\nResume Summary:\n", summary)
    print("\nExtracted Keywords:")
    for word, count in keywords:
        print(f" - {word} (count: {count})")

    print(f"\nResume Score: {score}/100")
    if suggestions:
        print("\nSuggestions to Improve:")
        for s in suggestions:
            print(f" - {s}")
    else:
        print("\nYour resume looks great!")

if __name__ == "__main__":
    resume_path = r"C:\Users\nares\Downloads\Resume.pdf" #Replace with your Original File path
    analyze_cv(resume_path)


Analyzing Resume:


Device set to use cpu



Resume Summary:
 I'm an enthusiastic AI & DS undergraduate with a passion for problem-solving and technology. I love exploringmachine learning, databases, and web development. I thrive in teamwork, communication, and critical thinking. Excited about opportunities that let me grow, contribute, and make an impact in the field of AI.

Extracted Keywords:
 - data (count: 4)
 - learning (count: 3)
 - machine (count: 3)
 - structures (count: 3)
 - using (count: 3)
 - ai (count: 2)
 - algorithms (count: 2)
 - basics (count: 2)
 - com (count: 2)
 - communication (count: 2)

Resume Score: 100/100

Your resume looks great!
